In [12]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import neighbors, linear_model
from sklearn.model_selection import cross_val_predict, KFold, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, f1_score, classification_report, make_scorer
from sklearn.preprocessing import PolynomialFeatures



data = pd.read_csv('data.csv')
song_names = data['song name']
song_names
data = data.drop(columns=['song name'])


<h1>Setting up </h1>

In [13]:
#data2 = data[data.genre != 'westcoastrap']
msfcc_names = []
for i in range(1, 21):
    msfcc_names.append('mfcc{0}'.format(i))
X = data[['zero crossing rate', 'spectral bandwidth', 'spectral centroid', 'chroma'] + msfcc_names]

In [14]:
genre_dict = {'alternative': 0, 'westcoastrap': 1,'edm': 2, 'jazz': 3, 'classical': 4} 
Y = [genre_dict[item] for item in data['genre']]

<h2>KNN-analysis</h2>

In [15]:
for k in range(1,11):
    model = neighbors.KNeighborsClassifier(k)
    yhat= cross_val_predict(model,X,Y, cv = StratifiedKFold(5, True))
    label_ord = range(5)
    Titanic_Conf_matrix = confusion_matrix(yhat,Y, labels=label_ord)
    print('{0}-nearest neighbor accuracy:{1}'.format(k, np.trace(Titanic_Conf_matrix)/np.sum(Titanic_Conf_matrix)))
    print(Titanic_Conf_matrix)

1-nearest neighbor accuracy:0.5799086757990868
[[62 11 12  6  0]
 [11 22  4  4  2]
 [15  7  2  0  0]
 [ 4  5  2  8  2]
 [ 4  1  0  2 33]]
2-nearest neighbor accuracy:0.5707762557077626
[[74 23 17  9  4]
 [10 20  2  3  0]
 [ 8  2  0  2  0]
 [ 3  1  1  4  6]
 [ 1  0  0  2 27]]
3-nearest neighbor accuracy:0.6164383561643836
[[77 19 15  8  2]
 [ 6 23  4  2  1]
 [ 6  2  0  1  1]
 [ 2  1  1  5  3]
 [ 5  1  0  4 30]]
4-nearest neighbor accuracy:0.6210045662100456
[[75 24 17  6  2]
 [ 8 19  0  2  0]
 [ 6  2  2  0  1]
 [ 3  0  1  8  2]
 [ 4  1  0  4 32]]
5-nearest neighbor accuracy:0.6164383561643836
[[75 23 17  8  2]
 [10 19  2  0  0]
 [ 2  2  0  0  0]
 [ 4  1  1  8  2]
 [ 5  1  0  4 33]]
6-nearest neighbor accuracy:0.6027397260273972
[[79 24 17  9  5]
 [ 7 18  2  2  1]
 [ 3  2  0  0  0]
 [ 2  1  1  5  1]
 [ 5  1  0  4 30]]
7-nearest neighbor accuracy:0.6027397260273972
[[77 28 18  8  0]
 [10 15  1  1  1]
 [ 2  2  0  0  1]
 [ 3  0  1  7  2]
 [ 4  1  0  4 33]]
8-nearest neighbor accuracy:0.6164

<h3>KDTree</h3>

In [16]:

#this code prints cluters of all songs that are similar
from sklearn.neighbors import KDTree

arr = np.array(X)

tree = KDTree(arr, leaf_size=4)

# r = 1.5 (means radius 1.5)
all_nn_indices  = tree.query_radius(arr, r=150)

all_nns = [[song_names[idx] for idx in nn_indices] for nn_indices in all_nn_indices]
for nns in all_nns:
    print(nns)

['Nujabes - Lady Brown.mp3', '02 Airplanes.m4a']
['Oh Wonder - Technicolour Beat.mp3', 'Ice Cube - You Know How We Do It (Official Video).mp3', 'Chet Baker - Tenderly.mp3']
["Eazy-E - It's On.mp3", "Compton's Most Wanted - Hood Took Me Under.mp3", 'Xzibit - X.mp3', 'Snoop doggy dogg - aint no fun (if the homies cant get none).mp3', 'flicker_porterRobinson.mp3', 'Miles Davis - Milestones (Audio).mp3', 'Coheed and Cambria Neverender.mp3']
['Arctic Monkeys - Teddy Picker (Official Video).mp3', 'WC - The Streets ft. Snoop Dogg, Nate Dogg.mp3', 'Dr. Dre - Still D.R.E. ft. Snoop Dogg.mp3', 'Stephen - Crossfire.mp3', '213 Groupie Luv.mp3']
['snoop doggy dogg - doggy dogg world.mp3', '2Pac - Skandalouz.mp3']
['OceanBlue_WRLD.mp3', 'Snoop Dogg - Midnight Love.mp3', '02 The Two Sides of Monsieur Valenti.m4a', '09 Bourgeois.m4a']
['snoop doggy dogg - doggy dogg world.mp3', 'Dr. Dre - Forgot About Dre ft. Eminem, Hittman.mp3']
['03 Crystalised.m4a', 'Ryo Fukui - It Could Happen To You.mp3', 'Kurup

<h2>Logistic Regression</h2>

This performs logistic regression with a number of different feature counts, cross validated with 5-fold CV.

Adding more dimensions did not produce significant results, but it was significantly better than K-Nearest Neighbors.

In [17]:
#logistic regression testing
for pval in range(1,4):
    model = linear_model.LogisticRegression()
    pf = PolynomialFeatures(degree = pval)
    pf.fit(X)
    yhat = cross_val_predict(model, pf.transform(X), Y, cv = StratifiedKFold(5,True))
    
    Titanic_Conf_matrix = confusion_matrix(yhat,Y)
    print("{0}-degree poly logistic regression accuracy:{1}".format(pval, np.trace(Titanic_Conf_matrix)/np.sum(Titanic_Conf_matrix)))
    print(Titanic_Conf_matrix)

1-degree poly logistic regression accuracy:0.7990867579908676
[[86  5  5  2  0]
 [ 2 33  4  5  0]
 [ 4  3 11  2  0]
 [ 3  3  0  9  1]
 [ 1  2  0  2 36]]


/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/py

2-degree poly logistic regression accuracy:0.8082191780821918
[[86  2  2  3  0]
 [ 4 36  6  4  0]
 [ 2  3 11  2  0]
 [ 4  3  1 10  3]
 [ 0  2  0  1 34]]


/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/py

3-degree poly logistic regression accuracy:0.7899543378995434
[[85  1  2  4  1]
 [ 2 34  5  4  0]
 [ 5  5  9  1  0]
 [ 4  5  4  9  0]
 [ 0  1  0  2 36]]


<h2>Experimentation on percentage of each type each song was</h2>

As one of the main goals of this was to be able to predict the genre of a song given a database, we perform leave-one-out cross-validation with logistic regression on the values we have.

In [20]:
print('perc_alt', 'perc_classical', 'perc_edm', 'perc_jazz', 'perc_rap')
def abx(Z,x):
    return np.append(Z[:x],Z[x+1:],axis=0)
pf = PolynomialFeatures(degree=1)
pf.fit(X)
phi = pf.transform(X)
score = 0
for idx in range(np.shape(phi)[0]):
    
    model = linear_model.LogisticRegression()
    # To test things, we fit on everything else
    model.fit(abx(phi,idx),abx(Y,idx))
    predicted_scores = model.predict_proba(phi[idx:idx+1])[0]
    # normalize percentages
    perc_alt = "%.6f" % ((predicted_scores[0] / sum(predicted_scores)) * 100)
    perc_rap = "%.6f" % ((predicted_scores[1] / sum(predicted_scores)) * 100)
    perc_edm = "%.6f" % ((predicted_scores[2] / sum(predicted_scores)) * 100)
    perc_jazz = "%.6f" % ((predicted_scores[3] / sum(predicted_scores)) * 100)
    perc_classical = "%.6f" % ((predicted_scores[4] / sum(predicted_scores)) * 100)
    score += predicted_scores[Y[idx]]
    print(song_names[idx])
    print(perc_alt, '    ', perc_classical, '       ', perc_edm, '    ', perc_jazz, '    ', perc_rap)
#order: 'alt', 'classical', 'edm', 'jazz'
print("overall score: {0}".format(score/np.shape(X)[0]))

perc_alt perc_classical perc_edm perc_jazz perc_rap
Nujabes - Lady Brown.mp3
1.440736      0.000052         27.752163      68.776329      2.030720
Ice Cube - You Know How We Do It (Official Video).mp3
0.811481      0.006456         0.007188      95.448876      3.726000
Xzibit - X.mp3
5.855544      0.000000         0.000463      0.081459      94.062534
WC - The Streets ft. Snoop Dogg, Nate Dogg.mp3
0.000764      0.000001         1.411328      0.133985      98.453922
2Pac - Skandalouz.mp3
0.450177      0.000000         0.141004      0.037125      99.371694
Snoop Dogg - Midnight Love.mp3
2.147774      0.000000         1.291948      13.986231      82.574048
Dr. Dre - Forgot About Dre ft. Eminem, Hittman.mp3
5.859086      0.000000         0.079585      0.133835      93.927494
Kurupt - We Can Freak It.mp3
7.864905      0.015692         0.067325      19.779791      72.272286
Cyne - Midas.mp3
53.591864      0.000000         2.185376      22.883068      21.339692
The Dogg Pound - I Don't Like t

/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/py

DJ Quik - Itz Your Fantasy.mp3
4.108069      0.000000         0.000286      0.432717      95.458927
2Pac - Happy Home (Nujabes Remix).mp3
1.089925      0.000909         15.803953      62.894440      20.210773
Gin and juice - Snoop Doggy Dogg.mp3
30.855900      0.000000         0.000002      0.000073      69.144026
Twinz - Eastside Lb.mp3
0.133783      0.000000         0.003807      0.592799      99.269611
The Game, 50 Cent - Hate It Or Love It (Official Music Video).mp3
0.392150      0.000000         0.085789      2.159056      97.363005
213 Groupie Luv.mp3
0.084328      0.000000         0.908705      0.091986      98.914981
Landon Wordswell - I think i need you now (prod by Kondor) - 2011 (Free Mixtape).mp3
0.053225      0.000000         50.165144      0.000438      49.781193
Ice Cube & Dr DRE - Natural Born Killaz.mp3
1.607263      0.000000         0.000015      0.535129      97.857593
South Central Cartel - All Day Everyday.mp3
0.070273      0.000000         0.000216      10.861738 

/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/py

Snoop doggy dogg - aint no fun (if the homies cant get none).mp3
0.031534      0.000000         0.007939      0.495576      99.464951
Dr. Dre - Still D.R.E. ft. Snoop Dogg.mp3
0.003442      1.009131         2.767418      3.015809      93.204201
Ice Cube - It Was A Good Day (Official Video).mp3
0.004438      35.221300         0.000060      0.728259      64.045944
N.W.A. - Straight Outta Compton.mp3
8.771753      0.000000         0.000173      0.051241      91.176833
Snoop Doggy Dogg - Murder Was The Case ( Uncensored ).mp3
0.213012      0.000000         48.571662      0.000347      51.214979
Warren G - Regulate ft. Nate Dogg.mp3
0.008310      0.000000         1.317701      16.696077      81.977912
Ice Cube - Ghetto Bird (Music Video).mp3
0.287231      0.000000         1.437863      0.871090      97.403817
Compton's Most Wanted - Hood Took Me Under.mp3
0.014535      0.000000         0.023272      0.105375      99.856818
Dr. Dre - Let Me Ride.mp3
0.068207      0.000000         0.001734   

...

/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/py

09 If You Were There, Beware.m4a
99.877140      0.000000         0.002165      0.117106      0.003590
09 Was It You_.m4a
99.157780      0.000000         0.000441      0.023147      0.818631
Red Hot Chili Peppers - Dark Necessities [OFFICIAL VIDEO].mp3
33.849326      0.000001         34.974528      31.129894      0.046252
04 Barely Legal.m4a
98.881515      0.000000         0.000075      1.118368      0.000042
03 Crystalised.m4a
93.272196      0.000007         5.853593      0.873406      0.000798
03 In the Aeroplane Over the Sea.m4a
99.937385      0.000000         0.014608      0.047988      0.000019
01 Entertainment.m4a
99.767340      0.000000         0.020658      0.211454      0.000548
10 Happy Ending.m4a
99.958812      0.000000         0.001637      0.039475      0.000076
07 Empty Party Rooms.m4a
99.784523      0.000000         0.061876      0.153017      0.000584
07 Do Me a Favour.m4a
99.609318      0.000000         0.001828      0.171444      0.217410
05 Bankrupt!.m4a
90.662501    

/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/py

09 Chances.m4a
97.478126      0.000000         0.007078      2.510248      0.004547
11 Colly Strings.m4a
99.928477      0.000000         0.027609      0.032827      0.011087
02 The Two Sides of Monsieur Valenti.m4a
99.779556      0.000000         0.129440      0.089980      0.001025
06 Drakkar Noir.m4a
99.965647      0.000000         0.032973      0.001275      0.000105
Oasis - Wonderwall (Official Video).mp3
62.150309      0.001732         0.024348      28.115643      9.707969
01 Wide Eyes.m4a
97.594703      0.000000         0.002534      2.081003      0.321760
02 Airplanes.m4a
95.695538      0.000003         0.698911      3.603185      0.002363
11 Legit Tattoo Gun.m4a
99.725537      0.000000         0.041460      0.232154      0.000849
03 D Is for Dangerous.m4a
99.997366      0.000000         0.001938      0.000393      0.000303
08 Archipelago.m4a
89.615701      0.000000         0.442689      8.532124      1.409486
overall score: 0.7621335117387228


/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ben/music_classifier/venv/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/ben/music_classifier/venv/lib/py